In [ ]:
import requests
import pandas as pd
import json
import psycopg2 as ps

In [ ]:
keys = json.load(open('keys.json'))
API_KEY = keys['API_KEY']


In [ ]:
pageSize = '10'
url = 'https://rebrickable.com/api/v3/lego/sets/?key=' + API_KEY + '&page_size=' + pageSize
response = requests.get(url)
results = response.json()['results']
df = pd.json_normalize(results)
df.head()

In [ ]:
def connect_to_db(host_name, dbname, username, password, port):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
    except ps.OperationalError as err:
        raise err
    else:
        print('Connected!')
        return conn

In [ ]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS sets (
                    set_num VARCHAR(255) PRIMARY KEY,
                    name TEXT,
                    year INTEGER,
                    theme_id INTEGER,
                    num_parts INTEGER,
                    set_img_url TEXT,
                    set_url TEXT,
                    last_modified_dt DATE
    )""")

    curr.execute(create_table_command)

In [ ]:
def check_if_row_exists(curr, set_num):
    query = ("""SELECT set_num FROM sets WHERE set_num = %s""")
    curr.execute(query, (set_num,))

    return curr.fetchone() is not None

In [ ]:
def update_row(curr, name, year, theme_id, num_parts, set_img_url, set_url, last_modified_dt):
    query = ("""UPDATE set_num
            SET name = %s,
                year = %s,
                theme_id = %s,
                num_parts = %s,
                set_img_url = %s,
                set_url = %s,
                last_modified_dt = %s
            WHERE set_num = %s""")

    vars_to_update = (name, year, theme_id, num_parts, set_img_url, set_url, last_modified_dt)
    curr.execute(query, vars_to_update)

In [ ]:
def update_db(curr, df):
    temp_df = pd.DataFrame(columns=['set_num', 'name', 'year', 'theme_id', 'num_parts', 'set_img_url', 'set_url', 'last_modified_dt'])

    for i, row in df.iterrows():
        if check_if_row_exists(curr, row['set_num']):
            update_row(curr, row['name'], row['year'], row['theme_id'], row['num_parts'], row['set_img_url'], row['set_url'], row['last_modified_dt'])
        else:
            temp_df = temp_df.append(row)
    return temp_df

In [ ]:
def insert_into_table(curr, set_num, name, year, theme_id, num_parts, set_img_url, set_url, last_modified_dt):
    insert_into_sets = (
        """INSERT INTO sets (set_num, name, year, theme_id, num_parts, set_img_url, set_url, last_modified_dt) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)""")

    rows_to_insert = (set_num, name, year, theme_id, num_parts,
                      set_img_url, set_url, last_modified_dt)
    curr.execute(insert_into_sets, rows_to_insert)


In [ ]:
def append_from_df_to_db(curr, df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['set_num'], row['name'], row['year'], row['theme_id'], row['num_parts'], row['set_img_url'], row['set_url'], row['last_modified_dt'])


In [ ]:
connection_data = json.load(open('connection_data.json'))
host_name = connection_data['host_name']
dbname = connection_data['dbname']
username = connection_data['username']
password = connection_data['password']
port = connection_data['port']
conn = None

conn = connect_to_db(host_name, dbname, username, password, port)

In [ ]:
curr = conn.cursor()

In [ ]:
create_table(curr)

In [ ]:
new_set_df = update_db(curr, df)

In [ ]:
append_from_df_to_db(curr,new_set_df)

In [ ]:
conn.commit()